In [ ]:
# For reading credentials from the .env file
import os
from dotenv import load_dotenv
# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
# For invocation of LLM with REST API
import requests, json
from ibm_cloud_sdk_core import IAMTokenManager

In [ ]:
# URL of the hosted LLMs is hardcoded because at this time all LLMs share the same endpoint
url = "https://us-south.ml.cloud.ibm.com"
def get_credentials():
    load_dotenv()
    # Update the global variables that will be used for authentication in another function
    globals()["api_key"] = os.getenv("api_key", None)
    globals()["watsonx_project_id"] = os.getenv("project_id", None)

In [ ]:
# The get_model function creates an LLM model object with the specified parameters
def get_model(model_type,max_tokens,min_tokens,decoding,temperature):
    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature
    }
    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials={
            "apikey": api_key,
            "url": url
        },
        project_id=watsonx_project_id
        )

    return model

In [ ]:
import pandas as pd
df = pd.read_csv('customer_support_tickets.csv')
#https://www.kaggle.com/datasets/suraj520/customer-support-ticket-dataset

In [ ]:
# selecting rows based on condition
dataframe = df.loc[df['Ticket Status'] == "Closed"]

In [ ]:
dataframe.shape

In [ ]:
dataframe[["Ticket Description","Resolution"]].head()

In [ ]:
import os
from sentence_transformers import SentenceTransformer
current_working_directory = os.getcwd()
model_path = os.path.join(current_working_directory, "all-MiniLM-L6-v2", "sentence_transformers")
model = SentenceTransformer(model_path, cache_folder=True)  # Set cache_folder to None


In [ ]:
import os
from sentence_transformers import SentenceTransformer

def load_sentence_transformer_model(model_name):
    model_path = os.path.join(".", model_name)
    #model_path = os.path.join(current_working_directory,model_name, "sentence_transformers")
    print(model_path)
    return SentenceTransformer(model_path)

#model_name = 'paraphrase-MiniLM-L6-v2'
model_name = 'all-MiniLM-L6-v2'
model = load_sentence_transformer_model(model_name)

In [ ]:
import os
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel

def load_sentence_transformer_model(model_name):
    current_working_directory = os.getcwd()
    model_path = os.path.join(current_working_directory, model_name, "sentence_transformers")

    if os.path.exists(model_path):
        return SentenceTransformer(model_path)
    else:
        try:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModel.from_pretrained(model_name)
            os.makedirs(model_path, exist_ok=True)
            tokenizer.save_pretrained(model_path)
            model.save_pretrained(model_path)
            return SentenceTransformer(model_path)
        except Exception as e:
            print("Error downloading the model from Hugging Face Hub:", e)
            raise

model_name = 'all-MiniLM-L6-v2'
model = load_sentence_transformer_model(model_name)


In [ ]:

import os
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
current_working_directory = os.getcwd()
def load_sentence_transformer_model(model_name):
    model_path = os.path.join(".", model_name)
    #model_path = os.path.join(current_working_directory,model_name, "sentence_transformers")
    # Check if the model exists locally
    if not os.path.exists(model_path):
        # Download the model using Hugging Face Hub
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        tokenizer.save_pretrained(model_path)
        model.save_pretrained(model_path)

    return SentenceTransformer(model_path)

#model_name = 'sentence-transformers/paraphrase-MiniLM-L6-v2'
model_name='all-MiniLM-L6-v2'
model = load_sentence_transformer_model(model_name)


In [ ]:

import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Create a pandas DataFrame
data = {'Ticket Description': ['Issue with the login', 'Cannot access the website', 'Error in the payment system'],
        'Resolution': ['Reset your password', 'Check your internet connection', 'Contact the support team']}

df = pd.DataFrame(data)

# Initialize the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create the vector database
ticket_descriptions = df['Ticket Description'].tolist()
ticket_vectors = model.encode(ticket_descriptions)

# Perform a similarity search for a given sentence
def search_ticket_resolution(query):
    query_vector = model.encode([query])
    similarities = util.pytorch_cos_sim(query_vector, ticket_vectors)
    most_similar_ticket_index = similarities[0].argmax()
    return df.iloc[most_similar_ticket_index]['Resolution']

query = "I can't log in to my account"
resolution = search_ticket_resolution(query)
print("Suggested Resolution:", resolution)


In [ ]:
loaders = [PyPDFLoader(file_path)]

index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)).from_loaders(loaders)

In [ ]:
def get_list_of_complaints():
    model_type = ModelTypes.MPT_7B_INSTRUCT2
    max_tokens = 100
    min_tokens = 50
    decoding = DecodingMethods.GREEDY
    # Temperature will be ignored if GREEDY is used
    temperature = 0.7
    # Instantiate the model
    model = get_model(model_type,max_tokens,min_tokens,decoding, temperature)
    complaint = f"""
            I just tried to book a flight on your incredibly slow website.  All 
            the times and prices were confusing.  I liked being able to compare 
            the amenities in economy with business class side by side.  But I 
            never got to reserve a seat because I didn't understand the seat map.  
            Next time, I'll use a travel agent!
            """
    prompt_get_complaints = f"""
    From the following customer complaint, extract 3 factors that caused the customer to be unhappy. 
    Put each factor on a new line. 
    Customer complaint:{complaint}
    Numbered list of all the factors that caused the customer to be unhappy:
    """
    # Invoke the model and print the results
    generated_response = model.generate(prompt=prompt_get_complaints)
    # WML API returns a dictionary object. Generated response is a list object that contains generated text
    # as well as several other items such as token count and seed
    # We recommmend that you put a breakpoint on this line and example the result object
    print("---------------------------------------------------------------------------")
    print("Prompt: " + prompt_get_complaints)
    print("List of complaints: " + generated_response['results'][0]['generated_text'])
    print("---------------------------------------------------------------------------")

In [ ]:
get_list_of_complaints()

In [ ]:
def answer_questions():
    final_prompt = "Write a paragraph about the capital of France."
    model_type = ModelTypes.FLAN_UL2
    max_tokens = 300
    min_tokens = 50
    decoding = DecodingMethods.SAMPLE
    temperature = 0.7
    # Instantiate the model
    model = get_model(model_type,max_tokens,min_tokens,decoding, temperature)
    # Invoke the model and print the results
    generated_response = model.generate(prompt=final_prompt)
    # WML API returns a dictionary object. Generated response is a list object that contains generated text

    print("---------------------------------------------------------------------------")
    print("Question/request: " + final_prompt)
    print("Answer: " + generated_response['results'][0]['generated_text'])
    print("---------------------------------------------------------------------------")

In [ ]:
def invoke_with_REST():
    rest_url ="https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-29"
    access_token = get_auth_token()
    model_type = "google/flan-ul2"
    max_tokens = 300
    min_tokens = 50
    decoding = "sample"
    temperature = 0.7

    final_prompt = "Write a paragraph about the capital of France."

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": "Bearer " + access_token
        }

    data = {
        "model_id": model_type,
        "input": final_prompt,
        "parameters": {
            "decoding_method": decoding,
            "max_new_tokens": max_tokens,
            "min_new_tokens": min_tokens,
            "temperature": temperature,
            "stop_sequences": ["."],
            },
        "project_id": watsonx_project_id
    }
    response = requests.post(rest_url, headers=headers, data=json.dumps(data))
    generated_response = response.json()['results'][0]['generated_text']

    print("--------------------------Invocation with REST-------------------------------------------")
    print("Question/request: " + final_prompt)
    print("Answer: " + generated_response)
    print("---------------------------------------------------------------------------")

In [ ]:
def get_auth_token():

    # Access token is required for REST invocation of the LLM
    access_token = IAMTokenManager(apikey=api_key,url="https://iam.cloud.ibm.com/identity/token").get_token()
    return access_token

def demo_LLM_invocation():
    # Load the api key and project id
    get_credentials()
    # Show examples of 2 use cases/prompts
    answer_questions()
    get_list_of_complaints()
    # Simple prompt - invoked with the REST API
    invoke_with_REST()
demo_LLM_invocation()

In [ ]:
# For reading credentials from the .env file
import os
from dotenv import load_dotenv

from sentence_transformers import SentenceTransformer
from chromadb.api.types import EmbeddingFunction

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods

import requests
from bs4 import BeautifulSoup
import spacy
import chromadb
import en_core_web_md

# Important: hardcoding the API key in Python code is not a best practice. We are using
# this approach for the ease of demo setup. In a production application these variables
# can be stored in an .env or a properties file

# URL of the hosted LLMs is hardcoded because at this time all LLMs share the same endpoint
url = "https://us-south.ml.cloud.ibm.com"

# These global variables will be updated in get_credentials() function
watsonx_project_id = ""
# Replace with your IBM Cloud key
api_key = ""

def get_credentials():

    load_dotenv()
    # Update the global variables that will be used for authentication in another function
    globals()["api_key"] = os.getenv("api_key", None)
    globals()["watsonx_project_id"] = os.getenv("project_id", None)

# The get_model function creates an LLM model object with the specified parameters

def get_model(model_type, max_tokens, min_tokens, decoding, temperature, top_k, top_p):
    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature,
        GenParams.TOP_K: top_k,
        GenParams.TOP_P: top_p,
    }

    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials={
            "apikey": api_key,
            "url": url
        },
        project_id=watsonx_project_id
    )

    return model


def get_model_test(model_type, max_tokens, min_tokens, decoding, temperature):
    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature
    }

    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials={
            "apikey": api_key,
            "url": url
        },
        project_id=watsonx_project_id
    )

    return model


# Embedding function
class MiniLML6V2EmbeddingFunction(EmbeddingFunction):
    MODEL = SentenceTransformer('all-MiniLM-L6-v2')

    def __call__(self, texts):
        return MiniLML6V2EmbeddingFunction.MODEL.encode(texts).tolist()


def extract_text(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract contents of <p> elements
            p_contents = [p.get_text() for p in soup.find_all('p')]

            # Print the contents of <p> elements
            print("\nContents of <p> elements: \n")
            for content in p_contents:
                print(content)
            raw_web_text = " ".join(p_contents)
            # remove \xa0 which is used in html to avoid words break acorss lines.
            cleaned_text = raw_web_text.replace("\xa0", " ")
            return cleaned_text

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")


def split_text_into_sentences(text):
    nlp = spacy.load("en_core_web_md")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    cleaned_sentences = [s.strip() for s in sentences]
    return cleaned_sentences


def create_embedding(url, collection_name):
    cleaned_text = extract_text(url)
    cleaned_sentences = split_text_into_sentences(cleaned_text)

    client = chromadb.Client()

    collection = client.get_or_create_collection(collection_name)

    # Upload text to chroma
    collection.upsert(
        documents=cleaned_sentences,
        metadatas=[{"source": str(i)} for i in range(len(cleaned_sentences))],
        ids=[str(i) for i in range(len(cleaned_sentences))],
    )

    return collection


def create_prompt(url, question, collection_name):
    # Create embeddings for the text file
    collection = create_embedding(url, collection_name)

    # query relevant information
    relevant_chunks = collection.query(
        query_texts=[question],
        n_results=5,
    )
    context = "\n\n\n".join(relevant_chunks["documents"][0])
    # Please note that this is a generic format. You can change this format to be specific to llama
    prompt = (f"{context}\n\nPlease answer the following question in one sentence using this "
              + f"text. "
              + f"If the question is unanswerable, say \"unanswerable\". Do not include information that's not relevant to the question."
              + f"Question: {question}")

    return prompt


def main():

    # Get the API key and project id and update global variables
    get_credentials()

    # Try diffrent URLs and questions
    url = "https://www.usbank.com/financialiq/manage-your-household/buy-a-car/own-electric-vehicles-learned-buying-driving-EVs.html"

    question = "What are the incentives for purchasing EVs?"
    # question = "What is the percentage of driving powered by hybrid cars?"
    # question = "Can an EV be plugged in to a household outlet?"
    collection_name = "test_web_RAG"

    answer_questions_from_web(api_key, watsonx_project_id, url, question, collection_name)


def answer_questions_from_web(request_api_key, request_project_id, url, question, collection_name):
    # Update the global variable
    globals()["api_key"] = request_api_key
    globals()["watsonx_project_id"] = request_project_id

    # Specify model parameters
    model_type = "meta-llama/llama-2-70b-chat"
    max_tokens = 100
    min_tokens = 50
    top_k = 50
    top_p = 1
    decoding = DecodingMethods.GREEDY
    temperature = 0.7

    # Get the watsonx model = try both options
    model = get_model(model_type, max_tokens, min_tokens, decoding, temperature, top_k, top_p)

    # Get the prompt
    complete_prompt = create_prompt(url, question, collection_name)

    # Let's review the prompt
    print("----------------------------------------------------------------------------------------------------")
    print("*** Prompt:" + complete_prompt + "***")
    print("----------------------------------------------------------------------------------------------------")

    generated_response = model.generate(prompt=complete_prompt)
    response_text = generated_response['results'][0]['generated_text']

    # Remove trailing white spaces
    response_text = response_text.strip()

    # print model response
    print("--------------------------------- Generated response -----------------------------------")
    print(response_text)
    print("*********************************************************************************************")

    return response_text


# Invoke the main function
if __name__ == "__main__":
    main()
